# Covid 10 data analysis
Data analysis of CountyUAs_cases_table data downloaded & timestamed daily from PHE Data hosted by ESRI at [ArcGIS.com](https://www.arcgis.com/home/item.html?id=b684319181f94875a6879bbc833ca3a6)

***
## Import python libraries and set file locations

In [1]:
### Import libraries
# For data analysis
import pandas as pd
import numpy as np

# For downloading data from www
import urllib.request

# For finding and using file paths
import glob, os  

# For outputting Markdown formatted stuff to this notebook
from IPython.display import display, Markdown

# For dates
import datetime

In [2]:
# Set file locations, using os.path.join to make the paths OS independent
onedrive_folder = os.environ['ONEDRIVECONSUMER']
root_folder = os.path.join(onedrive_folder,'Documents','COVID19')
data_folder = os.path.join(root_folder,'data')
lookup_folder = os.path.join(data_folder,'lookups')


In [3]:
# Set the prefix for the daily data files
daily_file_prefix = 'county_cases_daily'

# Set the name for the daily timeseries data file
daily_cases_file_name = 'daily_cases.csv'

# Set the name for the weekly timeseries data fil
weekly_cases_file_name = 'weekly_cases.csv'

***
## Get the ONS lookup tables
The PHE data is at Local Authority District level, these lookups are used for aggregating the data into larger areas (e.g. Greater Manchester / London) 

In [4]:
# create a dictionary to store the lookup table urls
lookup_urls = {}

# Local Authority District to Combined Authority (December 2019) Lookup in England
lookup_urls['LAD_to_CA_lookup'] = 'https://opendata.arcgis.com/datasets/db4f8bae6bfa41babfafea3ec8a38c0e_0.csv'

# Local Authority District to Region (April 2019) Lookup in England
lookup_urls['LAD_to_RGN_lookup'] = 'https://opendata.arcgis.com/datasets/3ba3daf9278f47daba0f561889c3521a_0.csv'

# Local Authority Districts (April 2019) Names and Codes in the United Kingdom
lookup_urls['LAD_name_lookup'] = 'https://opendata.arcgis.com/datasets/c3ddcd23a15c4d7985d8b36f1344b1db_0.csv'

# Local Authority District to County (April 2019) Lookup in England
lookup_urls['LAD_to_CTY_lookup'] = 'https://opendata.arcgis.com/datasets/79c993a10398400bb025a00849a43dc0_0.csv'

In [5]:
# Read the lookup tables into a dictionary if they are in the lookups folder, download them first if they are not 
lookup_dataframes = {}
for (lookup_name, url) in lookup_urls.items():
    lookup_filepath = os.path.join(lookup_folder,lookup_name + '.csv')
    try:
        lookup_dataframes[lookup_name] = pd.read_csv(lookup_filepath)
    except:
        urllib.request.urlretrieve(url, lookup_filepath)
        lookup_dataframes[lookup_name] = pd.read_csv(lookup_filepath)

In [6]:
# Display the fisrt few lines of each lookup table
for (lookup_name, df_lookup) in lookup_dataframes.items():
    display(Markdown('***\n'+lookup_name))
    display(df_lookup.head())

***
LAD_to_CA_lookup

,LAD19CD,LAD19NM,CAUTH19CD,CAUTH19NM,column4,FID
0,E08000001,Bolton,E47000001,Greater Manchester,NaN,1
1,E08000002,Bury,E47000001,Greater Manchester,NaN,2
2,E08000003,Manchester,E47000001,Greater Manchester,NaN,3
3,E08000004,Oldham,E47000001,Greater Manchester,NaN,4
4,E08000005,Rochdale,E47000001,Greater Manchester,NaN,5


***
LAD_to_RGN_lookup

,LAD19CD,LAD19NM,RGN19CD,RGN19NM,FID
0,E09000001,City of London,E12000007,London,1
1,E06000054,Wiltshire,E12000009,South West,2
2,E09000002,Barking and Dagenham,E12000007,London,3
3,E09000003,Barnet,E12000007,London,4
4,E09000004,Bexley,E12000007,London,5


***
LAD_name_lookup

,LAD19CD,LAD19NM,LAD19NMW,FID
0,E06000001,Hartlepool,NaN,1
1,E07000215,Tandridge,NaN,2
2,E07000084,Basingstoke and Deane,NaN,3
3,E06000002,Middlesbrough,NaN,4
4,E06000003,Redcar and Cleveland,NaN,5


***
LAD_to_CTY_lookup

,LAD19CD,LAD19NM,CTY19CD,CTY19NM,FID
0,E07000120,Hyndburn,E10000017,Lancashire,1
1,E07000121,Lancaster,E10000017,Lancashire,2
2,E07000122,Pendle,E10000017,Lancashire,3
3,E07000123,Preston,E10000017,Lancashire,4
4,E07000124,Ribble Valley,E10000017,Lancashire,5


### Create a lookup for the LAD code & name

In [7]:
# Create a lookup for the LAD code & name
df_LAD_name_lookup = lookup_dataframes['LAD_name_lookup'].reset_index()[['LAD19CD','LAD19NM']].set_index('LAD19CD').drop_duplicates()

In [8]:
df_LAD_name_lookup.head()

,LAD19NM
LAD19CD,
E06000001,Hartlepool
E07000215,Tandridge
E07000084,Basingstoke and Deane
E06000002,Middlesbrough
E06000003,Redcar and Cleveland


***
## Read the timestamped csv data files and combine them into a single timeseries

### Read the csvs

In [9]:
# Search for all the data csvs and create a list of the file paths
file_search_string = daily_file_prefix + "*.csv"
data_files = glob.glob(os.path.join(data_folder, file_search_string))

# Read each csv into a list of dataframes, then combine the list of dataframes into one dataframe
df_from_each_file = (pd.read_csv(f,index_col=0) for f in data_files)
df_combined = pd.concat(df_from_each_file, ignore_index=True,sort=False)


In [10]:
df_combined.head()

,GSS_CD,GSS_NM,TotalCases,time_stamp
0,E09000002,Barking and Dagenham,14,2020-03-19 13:55:00.000000
1,E09000003,Barnet,27,2020-03-19 13:55:00.000000
2,E08000016,Barnsley,6,2020-03-19 13:55:00.000000
3,E06000022,Bath and North East Somerset,3,2020-03-19 13:55:00.000000
4,E06000055,Bedford,1,2020-03-19 13:55:00.000000


### Get the actual day the data refers to from the time_stamp
The data is collected the day after it is released, so subtract 1 day from the time_stamp

In [11]:
# Convert the 'time_stamp' column to pandas datetime
df_combined['time_stamp']=pd.to_datetime(df_combined['time_stamp'])

In [12]:
# Check that the time_stamp datatype is a pandas datetime (datetime64[ns])
df_combined.dtypes

GSS_CD                object
GSS_NM                object
TotalCases             int64
time_stamp    datetime64[ns]
dtype: object

In [13]:
# Add a column for the year
df_combined['year']=df_combined['time_stamp'].dt.year

# Add a column for the day of the year
df_combined['day']=df_combined['time_stamp'].dt.dayofyear - 1

# Combine the Year
df_combined['date']=pd.to_datetime(df_combined['year'] * 1000 + df_combined['day'], format='%Y%j')

In [14]:
# Remove the redundant 'year' and 'time_stamp' columns
df_combined.drop(['year','time_stamp'],axis=1,inplace=True)

In [15]:
df_combined.head()

,GSS_CD,GSS_NM,TotalCases,day,date
0,E09000002,Barking and Dagenham,14,78,2020-03-18
1,E09000003,Barnet,27,78,2020-03-18
2,E08000016,Barnsley,6,78,2020-03-18
3,E06000022,Bath and North East Somerset,3,78,2020-03-18
4,E06000055,Bedford,1,78,2020-03-18


## Find the LAD / County codes for the data
'GSS_CD' sometimes refers to an LAD, and sometimes to a county. 

In [16]:
# Show the first few lines of the LAD_name_lookup
df_LAD_name_lookup.head()

,LAD19NM
LAD19CD,
E06000001,Hartlepool
E07000215,Tandridge
E07000084,Basingstoke and Deane
E06000002,Middlesbrough
E06000003,Redcar and Cleveland


In [17]:
# Get a list of all the GSS codes referenced in the data
GSS_codes_in_data = df_combined.reset_index()['GSS_CD'].drop_duplicates()

In [18]:
# Get the number of LADs listed in the LAD name lookup
number_of_LADs_in_total = len(df_LAD_name_lookup)

In [19]:
# Get a dataframe containing all the LADs not referenced in the data
df_LADs_not_in_data = df_LAD_name_lookup.loc[~df_LAD_name_lookup.index.isin(GSS_codes_in_data)]

In [20]:
# Get the number of GSS codes referenced in the data
number_of_GSS_codes_in_data = len(GSS_codes_in_data)

In [21]:
number_of_LADs_not_in_data = len(df_LADs_not_in_data)

In [22]:
number_of_LADs_in_total

382

In [23]:
number_of_GSS_codes_in_data

149

In [24]:
number_of_LADs_not_in_data

259

In [25]:
number_of_GSS_codes_in_data + number_of_LADs_not_in_data

408

In [26]:
# Get a series containing all the GSS references in the data that are found in the LAD lookup
df_GSS_is_LAD = GSS_codes_in_data[GSS_codes_in_data.isin(df_LAD_name_lookup.index)]

In [27]:
# Get a series containing all the GSS references in the data that not found in the LAD lookup
df_GSS_is_CTY = GSS_codes_in_data[~GSS_codes_in_data.isin(df_LAD_name_lookup.index)]

In [28]:
df_LAD_to_CTY_lookup = lookup_dataframes['LAD_to_CTY_lookup']

In [29]:
df_LAD_to_CTY_lookup.head()

,LAD19CD,LAD19NM,CTY19CD,CTY19NM,FID
0,E07000120,Hyndburn,E10000017,Lancashire,1
1,E07000121,Lancaster,E10000017,Lancashire,2
2,E07000122,Pendle,E10000017,Lancashire,3
3,E07000123,Preston,E10000017,Lancashire,4
4,E07000124,Ribble Valley,E10000017,Lancashire,5


In [30]:
df_combined.set_index('GSS_CD',inplace=True)

In [31]:
df_combined.head()

,GSS_NM,TotalCases,day,date
GSS_CD,,,,
E09000002,Barking and Dagenham,14,78,2020-03-18
E09000003,Barnet,27,78,2020-03-18
E08000016,Barnsley,6,78,2020-03-18
E06000022,Bath and North East Somerset,3,78,2020-03-18
E06000055,Bedford,1,78,2020-03-18


In [32]:
# Add a column to show if the data was collected at LAD level or county level
df_combined['area_type'] = np.where(df_combined.index.isin(df_GSS_is_LAD), 'LAD', 'CTY')


In [33]:
df_combined[df_combined['area_type']==False].head()

,GSS_NM,TotalCases,day,date,area_type
GSS_CD,,,,,


In [34]:
# Join with LAD_to_CTY_lookup to add county to data that is at LAD level (i.e. the GSS code refers to an LAD)
df_combined_county = df_combined.join(df_LAD_to_CTY_lookup[['LAD19CD','CTY19CD','CTY19NM']].set_index('LAD19CD'))
df_combined_county.index.rename('GSS_CD',inplace=True)

In [35]:
df_combined_county.reset_index(inplace=True)

In [36]:
# If the GSS code is at CTY level, use the GSS code / name for the CTY code / name
df_combined_county['CTY19CD'] = np.where(df_combined_county['GSS_CD'].isin(df_GSS_is_CTY), df_combined_county['GSS_CD'],df_combined_county['CTY19CD'])
df_combined_county['CTY19NM'] = np.where(df_combined_county['GSS_CD'].isin(df_GSS_is_CTY), df_combined_county['GSS_NM'],df_combined_county['CTY19NM'])

Some LADs aren't in the LAD_to_CTY_lookup:

In [37]:
df_combined_county[df_combined_county['CTY19CD'].isna()].head()

,GSS_CD,GSS_NM,TotalCases,day,date,area_type,CTY19CD,CTY19NM
0,E06000001,Hartlepool,1,78,2020-03-18,LAD,NaN,NaN
1,E06000001,Hartlepool,2,79,2020-03-19,LAD,NaN,NaN
2,E06000001,Hartlepool,2,80,2020-03-20,LAD,NaN,NaN
3,E06000001,Hartlepool,2,81,2020-03-21,LAD,NaN,NaN
4,E06000001,Hartlepool,2,82,2020-03-22,LAD,NaN,NaN


In [38]:
df_combined_LAD_county = df_combined_county.copy()

In [39]:
df_combined_LAD_county['LAD19CD'] = np.where(df_combined_LAD_county['GSS_CD'].isin(df_GSS_is_LAD), df_combined_LAD_county['GSS_CD'],np.NaN)
df_combined_LAD_county['LAD19NM'] = np.where(df_combined_LAD_county['GSS_CD'].isin(df_GSS_is_LAD), df_combined_LAD_county['GSS_NM'],np.NaN)

In [40]:
df_combined_LAD_county.head()

,GSS_CD,GSS_NM,TotalCases,day,date,area_type,CTY19CD,CTY19NM,LAD19CD,LAD19NM
0,E06000001,Hartlepool,1,78,2020-03-18,LAD,NaN,NaN,E06000001,Hartlepool
1,E06000001,Hartlepool,2,79,2020-03-19,LAD,NaN,NaN,E06000001,Hartlepool
2,E06000001,Hartlepool,2,80,2020-03-20,LAD,NaN,NaN,E06000001,Hartlepool
3,E06000001,Hartlepool,2,81,2020-03-21,LAD,NaN,NaN,E06000001,Hartlepool
4,E06000001,Hartlepool,2,82,2020-03-22,LAD,NaN,NaN,E06000001,Hartlepool


### Index the data

In [41]:
# Index the data by the LAD code and the timestamp, then sort the data
df_daily_cases = df_combined_LAD_county.set_index(['GSS_CD','date'])
df_daily_cases = df_daily_cases.sort_index()

In [42]:
df_daily_cases.head()

GSS_NM  TotalCases  day area_type CTY19CD CTY19NM  \
GSS_CD    date                                                                
E06000001 2020-03-18  Hartlepool           1   78       LAD     NaN     NaN   
          2020-03-19  Hartlepool           2   79       LAD     NaN     NaN   
          2020-03-20  Hartlepool           2   80       LAD     NaN     NaN   
          2020-03-21  Hartlepool           2   81       LAD     NaN     NaN   
          2020-03-22  Hartlepool           2   82       LAD     NaN     NaN   

                        LAD19CD     LAD19NM  
GSS_CD    date                               
E06000001 2020-03-18  E06000001  Hartlepool  
          2020-03-19  E06000001  Hartlepool  
          2020-03-20  E06000001  Hartlepool  
          2020-03-21  E06000001  Hartlepool  
          2020-03-22  E06000001  Hartlepool

***
# Calculate the change in TotalCases (velocity) and change in velocity (acceleration)

In [43]:
# Calculate the velocity of the change in the number of cases from the previous time

df_daily_cases['velocity'] = df_daily_cases['TotalCases'].groupby(level=0).diff()
df_daily_cases['acceleration'] = df_daily_cases['velocity'].groupby(level=0).diff()
df_daily_cases.head()

GSS_NM  TotalCases  day area_type CTY19CD CTY19NM  \
GSS_CD    date                                                                
E06000001 2020-03-18  Hartlepool           1   78       LAD     NaN     NaN   
          2020-03-19  Hartlepool           2   79       LAD     NaN     NaN   
          2020-03-20  Hartlepool           2   80       LAD     NaN     NaN   
          2020-03-21  Hartlepool           2   81       LAD     NaN     NaN   
          2020-03-22  Hartlepool           2   82       LAD     NaN     NaN   

                        LAD19CD     LAD19NM  velocity  acceleration  
GSS_CD    date                                                       
E06000001 2020-03-18  E06000001  Hartlepool       NaN           NaN  
          2020-03-19  E06000001  Hartlepool       1.0           NaN  
          2020-03-20  E06000001  Hartlepool       0.0          -1.0  
          2020-03-21  E06000001  Hartlepool       0.0           0.0  
          2020-03-22  E06000001  Hartlepool       0.0           0.0

## Save the data to csv

In [44]:
df_daily_cases.to_csv(daily_cases_file_name)

***
# Aggregate daily data into weekly

## Agregate the data

In [45]:
df_daily_cases.head()

GSS_NM  TotalCases  day area_type CTY19CD CTY19NM  \
GSS_CD    date                                                                
E06000001 2020-03-18  Hartlepool           1   78       LAD     NaN     NaN   
          2020-03-19  Hartlepool           2   79       LAD     NaN     NaN   
          2020-03-20  Hartlepool           2   80       LAD     NaN     NaN   
          2020-03-21  Hartlepool           2   81       LAD     NaN     NaN   
          2020-03-22  Hartlepool           2   82       LAD     NaN     NaN   

                        LAD19CD     LAD19NM  velocity  acceleration  
GSS_CD    date                                                       
E06000001 2020-03-18  E06000001  Hartlepool       NaN           NaN  
          2020-03-19  E06000001  Hartlepool       1.0           NaN  
          2020-03-20  E06000001  Hartlepool       0.0          -1.0  
          2020-03-21  E06000001  Hartlepool       0.0           0.0  
          2020-03-22  E06000001  Hartlepool       0.0           0.0

In [46]:
# Aggregate the daily TotalCases to get the weekly (Monday) maximum
df_GSS_max_weekly_cases = df_daily_cases.reset_index().groupby(['GSS_CD', pd.Grouper(key='date', freq='W-MON')])['TotalCases'].max()

# Aggregate the daily velocity & acceleration to get weekly (Monday) average
df_GSS_mean_weekly_rates = df_daily_cases.reset_index().groupby(['GSS_CD', pd.Grouper(key='date', freq='W-MON')])[['velocity','acceleration']].mean()

In [47]:
# Join the max cases and mean rates back together
df_GSS_CD_weekly_cases = df_GSS_max_weekly_cases.to_frame().join(df_GSS_mean_weekly_rates)

In [48]:
df_GSS_to_LAD_CTY = df_daily_cases.reset_index().set_index('GSS_CD')[['area_type','GSS_NM','CTY19CD','CTY19NM','LAD19CD','LAD19NM']]

In [49]:
# Add the LAD / CTY codes & names back in
df_GSS_weekly_cases = df_GSS_CD_weekly_cases.reset_index().set_index('GSS_CD').join(df_GSS_to_LAD_CTY)

In [50]:
# Rename the columns
columns={'date':'week_starting','TotalCases':'max_cases','velocity':'mean_velocity','acceleration':'mean_acceleration'}
df_GSS_weekly_cases.rename(columns=columns, inplace=True)

In [51]:
# Index the data by the LAD code and the timestamp, then sort the data
df_GSS_weekly_cases = df_GSS_weekly_cases.reset_index().set_index(['GSS_CD','week_starting'])
df_GSS_weekly_cases = df_GSS_weekly_cases.sort_index()

In [52]:
df_GSS_weekly_cases.head()

max_cases  mean_velocity  mean_acceleration  \
GSS_CD    week_starting                                                
E06000001 2020-03-23             3            0.4                0.0   
          2020-03-23             3            0.4                0.0   
          2020-03-23             3            0.4                0.0   
          2020-03-23             3            0.4                0.0   
          2020-03-23             3            0.4                0.0   

                        area_type      GSS_NM CTY19CD CTY19NM    LAD19CD  \
GSS_CD    week_starting                                                    
E06000001 2020-03-23          LAD  Hartlepool     NaN     NaN  E06000001   
          2020-03-23          LAD  Hartlepool     NaN     NaN  E06000001   
          2020-03-23          LAD  Hartlepool     NaN     NaN  E06000001   
          2020-03-23          LAD  Hartlepool     NaN     NaN  E06000001   
          2020-03-23          LAD  Hartlepool     NaN     NaN  E06000001   

                            LAD19NM  
GSS_CD    week_starting              
E06000001 2020-03-23     Hartlepool  
          2020-03-23     Hartlepool  
          2020-03-23     Hartlepool  
          2020-03-23     Hartlepool  
          2020-03-23     Hartlepool

In [53]:
# Write the data to a csv
df_GSS_weekly_cases.to_csv(weekly_cases_file_name)